In [1]:
import pandas as pd

In [2]:
# Target id
target_id = "CHEMBL2760"

In [3]:
from loader.load_chembl import parse_activities
# Load the data
target_raw = parse_activities(target_id=target_id)

In [4]:
target_raw.head()

,action_type,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,...,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
0,NaN,NaN,188231,[],CHEMBL644937,Inhibitory activity against Acid sphingomyelinase,B,NaN,NaN,BAO_0000190,...,Homo sapiens,Sphingomyelin phosphodiesterase,9606,NaN,NaN,IC50,ug ml-1,UO_0000274,NaN,5.25
1,NaN,NaN,194117,[],CHEMBL644937,Inhibitory activity against Acid sphingomyelinase,B,NaN,NaN,BAO_0000190,...,Homo sapiens,Sphingomyelin phosphodiesterase,9606,NaN,NaN,IC50,ug ml-1,UO_0000274,NaN,24.00
2,NaN,NaN,206131,[],CHEMBL644937,Inhibitory activity against Acid sphingomyelinase,B,NaN,NaN,BAO_0000190,...,Homo sapiens,Sphingomyelin phosphodiesterase,9606,NaN,NaN,IC50,ug ml-1,UO_0000274,NaN,5.30
3,NaN,NaN,206133,[],CHEMBL644937,Inhibitory activity against Acid sphingomyelinase,B,NaN,NaN,BAO_0000190,...,Homo sapiens,Sphingomyelin phosphodiesterase,9606,NaN,NaN,IC50,ug ml-1,UO_0000274,NaN,5.15
4,NaN,NaN,210721,[],CHEMBL644937,Inhibitory activity against Acid sphingomyelinase,B,NaN,NaN,BAO_0000190,...,Homo sapiens,Sphingomyelin phosphodiesterase,9606,NaN,NaN,IC50,ug ml-1,UO_0000274,NaN,34.50


In [5]:
from loader.clean_chembl import clean_data
# Clean the data
target_clean = clean_data(target_id=target_id, rewrite=False)

In [6]:
target_clean.head()

,canonical_smiles,standard_value
molecule_chembl_id,,
CHEMBL323197,COc1c(O)cc2oc3cc(O)c(CC=C(C)C)c(O)c3c(=O)c2c1C...,14100.0
CHEMBL4167681,CCCCCCCCCCOc1cc(C(=O)NO)ccc1OC,890.0
CHEMBL320658,COc1c(O)cc(O)c(C(=O)c2c(O)cc(O)cc2O)c1CC=C(C)C,14700.0
CHEMBL109045,COc1cc(C(=O)c2c(O)cc(O)c(CC=C(C)C)c2O)c(O)cc1O,66000.0
CHEMBL4464068,CCCCCCCCCCOc1cc(C(=O)NO)on1,360.0


In [7]:
from filter.filter_molecules import filter_molecules

df_filtered = filter_molecules(target_clean)

<frozen importlib._bootstrap>:241: RuntimeWarning: to-Python converter for class boost::shared_ptr<class RDKit::FilterHierarchyMatcher> already registered; second conversion method ignored.


In [8]:
df_filtered

,molecule_chembl_id,smiles,ic50,tox_free,bbb,qed,sa,lip,carc
